In [1]:
import torch
from torch import nn
import time
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from sklearn import metrics
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### LR using pytorch

In [2]:
device = torch.device(0)
torch.cuda.get_device_name(0)

'GeForce GTX 1060'

In [3]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = torch.Tensor(x_train).view(-1,28*28).to(device)
x_test = torch.Tensor(x_test).view(-1,28*28).to(device)
y_train = torch.Tensor(y_train).to(device).long()
y_test = torch.Tensor(y_test).to(device).long()

In [4]:
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.Linear = nn.Linear(input_size, num_classes)
        
    def forward(self,x):
        return self.Linear(x)
    
    def predict(self,x):
        out = model(x)
        _, out = torch.max(out,1)
        return out

In [5]:
input_size = 28*28
num_classes = 10
lr = 0.0001

model = LogisticRegression(input_size,num_classes).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr)
for epoch in range(100):
    outputs = model(x_train)
    loss = criterion(outputs,y_train)
    loss.backward()
    optimizer.step()

In [6]:
metrics.accuracy_score(y_test.cpu(),model.predict(x_test).cpu())

0.8949

In [7]:
metrics.confusion_matrix(y_test.cpu(),model.predict(x_test).cpu())

array([[ 951,    0,    4,    4,    0,   11,    4,    3,    2,    1],
       [   0, 1078,    6,   11,    0,    3,    3,    1,   32,    1],
       [   8,    1,  923,   16,   14,    6,    9,   21,   24,   10],
       [   5,    0,   31,  878,    1,   41,    2,   10,   25,   17],
       [   2,    0,    3,    3,  901,    0,    8,    3,   16,   46],
       [  13,    3,    7,   32,   12,  758,   16,    6,   37,    8],
       [  24,    2,   10,    5,   29,   14,  869,    1,    3,    1],
       [   4,    6,   26,   16,    7,    0,    0,  916,    4,   49],
       [  10,    3,   22,   46,   16,   61,    8,   13,  770,   25],
       [   4,    4,    3,    9,   41,   12,    0,   22,    9,  905]],
      dtype=int64)

### LR using sklearn

In [8]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train,x_test = x_train.reshape(-1,28*28), x_test.reshape(-1,28*28)
model = linear_model.LogisticRegression(multi_class='auto', solver='newton-cg', n_jobs=4)
model.fit(x_train[:1000],y_train[:1000])
pred = model.predict(x_test)
metrics.accuracy_score(y_test,pred)

0.8468

In [9]:
metrics.confusion_matrix(y_test,pred)

array([[ 931,    0,   11,    4,    0,    4,   11,    9,    6,    4],
       [   0, 1096,    2,    2,    1,    2,    5,    3,   23,    1],
       [   9,   11,  895,   19,   11,    2,   21,   16,   38,   10],
       [   6,    1,   54,  761,    3,  109,    8,   23,   25,   20],
       [   3,    2,   10,   10,  826,    5,   20,    3,   14,   89],
       [  21,    6,   15,   31,   21,  687,   21,   22,   46,   22],
       [  11,    6,   30,    0,   11,   44,  845,    1,   10,    0],
       [   2,   13,   34,   19,    6,    7,    1,  898,    1,   47],
       [  13,   20,   20,   16,   14,   58,   24,   31,  739,   39],
       [  10,    5,   11,   30,   46,   26,    2,   85,    4,  790]],
      dtype=int64)

### CNN using pytorch

In [10]:
device = torch.device(0)
torch.cuda.get_device_name(0)

(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = torch.Tensor(x_train).view(-1,1,28,28).to(device)
x_test = torch.Tensor(x_test).view(-1,1,28,28).to(device)
y_train = torch.Tensor(y_train).to(device).long()
y_test = torch.Tensor(y_test).to(device).long()

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,4,5)
        self.conv2 = nn.Conv2d(4,10,5)
        self.fc1 = nn.Linear(10*4*4, 60)
        self.fc2 = nn.Linear(60,10)
        
    def forward(self,x):
        x = F.relu(self.conv1(x)) #4,24,24
        x = F.max_pool2d(x,2)     #4,12,12
        x = F.relu(self.conv2(x)) #10,8,8
        x = F.max_pool2d(x,2)     #10,4,4
        x = x.view(-1,10*4*4)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def predict(self,x):
        out = model(x)
        _, out = torch.max(out,1)
        return out

In [12]:
lr = 0.001
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(x_train)
    loss = criterion(outputs,y_train)
    loss.backward()
    optimizer.step()
    if epoch%5==0:
        pred = model.predict(x_test)
        precision = metrics.accuracy_score(y_test.cpu(),pred.cpu())
        print(epoch, round(100*precision,2),'%')

0 11.83 %
5 23.28 %
10 34.5 %
15 44.03 %
20 59.69 %
25 67.33 %
30 75.44 %
35 80.72 %
40 83.6 %
45 85.33 %
50 86.78 %
55 87.68 %
60 88.58 %
65 89.12 %
70 89.82 %
75 90.25 %
80 90.47 %
85 90.73 %
90 90.97 %
95 91.25 %


In [13]:
metrics.confusion_matrix(y_test.cpu(),pred.cpu())

array([[ 944,    0,    4,    1,    0,    8,   14,    3,    5,    1],
       [   0, 1102,    7,    2,    4,    0,    3,    1,   16,    0],
       [   9,    3,  909,   27,   16,    9,   13,   15,   18,   13],
       [  11,    2,   29,  902,    0,   32,    6,   11,   14,    3],
       [   4,    4,    2,    0,  903,    4,   14,    2,    8,   41],
       [  15,    3,    2,   30,    6,  770,   15,    7,   34,   10],
       [  13,    5,    7,    1,    9,   25,  886,    2,    9,    1],
       [   1,    9,   35,    9,    7,    6,    0,  920,    6,   35],
       [   4,    4,   11,   22,    5,   11,    8,   10,  881,   18],
       [   9,    9,    1,    8,   30,   14,    1,   22,    7,  908]],
      dtype=int64)